In [ ]:
import numpy as np

from simul.vis.signals import get_vis_df, vis_signals, vis_signals2d, get_fft_df


In [ ]:
# Freq - 1Hz, sample rate - 1ms
sample_rate_full = 1000  #  Samples per s
sin_freq = 1  # Hz
sin_omega = (2*np.pi)*sin_freq
sin_ts = np.arange(0, 20, 1/sample_rate_full)

signal_vals_full = np.exp(1j * sin_omega * sin_ts)

df = get_vis_df(
    sin_ts,
    np.array([signal_vals_full]),
    np.array([signal_vals_full]),
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)

In [ ]:
def prune_normal(signal, rate=0.05, std=0.1):
#     ratio: 0.1
    res = np.full(signal.shape, np.NaN, dtype=complex)
    grid_ids = np.arange(0, signal.shape[0], 1/rate)
    grid_deltas = np.random.normal(0, std, grid_ids.shape)
    grid_ids += grid_deltas
    grid_ids = grid_ids[(grid_ids < signal.shape[0]-1)&((grid_ids > 0))]
#     print(np.rint(grid_ids).astype(int))
    res[np.rint(grid_ids).astype(int)] = signal[np.rint(grid_ids).astype(int)]
    return res

def prune_uniform(signal, interval, spread, sample_rate=1000):
    res = np.full(signal.shape, np.NaN, dtype=complex)
    
    grid_ids = np.arange(0, signal.shape[0], interval*sample_rate)

    grid_deltas = np.random.uniform(-sample_rate*spread/2, sample_rate*spread/2,
                                    grid_ids.shape)
    grid_ids += grid_deltas
    grid_ids = grid_ids[(grid_ids < signal.shape[0]-1)&((grid_ids > 0))]

    res[np.rint(grid_ids).astype(int)] = signal[np.rint(grid_ids).astype(int)]
    return res


In [ ]:

# signal_vals_pruned = prune_normal(signal_vals_full, rate = 0.1, std=2)
signal_vals_pruned = prune_uniform(signal_vals_full, interval = 0.1, spread=0.1)
signal_vals_zero_filled = np.nan_to_num(signal_vals_pruned, copy=True, nan=0)

df = get_vis_df(
    sin_ts,
    np.array([signal_vals_full]),
    np.array([signal_vals_pruned]),
#     np.array([signal_vals_zero_filled]),
#     (np.array([signal_vals_zero_filled]), "zero_filled", ),
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)

In [ ]:
m = np.nanmean(signal_vals_pruned)
a = np.array(signal_vals_pruned)
# np.nan_to_num(np.array(signal_vals_pruned, dtype=np.complex128), nan=m)
a[np.isnan(a)] = m
a

In [ ]:
from scipy import signal
import itertools

def iir_lowpass(cutoff, fs, order=10):
#     return signal.iirfilter(order, cutoff, fs=fs, btype='low', analog=False)
    return signal.iirfilter(order, cutoff*0.9, btype='low', analog=False, ftype="butter")

def iir_lowpass_filter(data, cutoff, fs, order=5, doublepass=True):
    b, a = iir_lowpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data) if doublepass else signal.lfilter(b, a, data)
#     y = signal.lfilter(b, a, data)

    return y


def iir_lowpass_interp(signal_vals , order = 5):
    t_rate = 1

    sample_rate = 1 / max([len(list(g)) for k, g in itertools.groupby(signal_vals, lambda x: np.isnan(x)) if k])

    vals_zero_filled = np.nan_to_num(signal_vals, copy=True, nan=0)
    
    b, a = iir_lowpass(sample_rate/2, t_rate, order)
    vals_interp = iir_lowpass_filter(vals_zero_filled, sample_rate/2, t_rate, order)
    norm_coeff = np.nanmean(np.abs(signal_vals))/np.mean(np.abs(vals_interp))
    return vals_interp*norm_coeff

def iir_lowpass_interp_fill_means(signal_vals , order = 5):
    t_rate = 1
    sample_rate = 1 / max([len(list(g)) for k, g in itertools.groupby(signal_vals, lambda x: np.isnan(x)) if k])

    vals_filled = np.array(signal_vals)
    fill_with = np.nanmean(vals_filled)
    vals_filled[np.isnan(vals_filled)] = fill_with
    print(fill_with)
    b, a = iir_lowpass(sample_rate/2, t_rate, order)
    vals_interp = iir_lowpass_filter(vals_filled, sample_rate/2, t_rate, order)
#     norm_coeff = np.nanmean(np.abs(signal_vals))/np.mean(np.abs(vals_interp))
    norm_coeff = 1
    return vals_interp*norm_coeff




In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

def show_filter_reponse(b, a, cutoff):
    w, h = signal.freqs(b, a)
    px.line(x=w, y=np.abs(h), log_x=True).add_vline(x=cutoff).show()

cutoff = 10
b, a = signal.iirfilter(10, cutoff*0.9, btype='low', analog=True, ftype="butter")
show_filter_reponse(b, a, cutoff)

In [ ]:
df = get_vis_df(
    sin_ts,
    np.array([signal_vals_full]),
    
    np.array([signal_vals_pruned]),
#     np.array([signal_vals_zero_filled]),
    (np.array([iir_lowpass_interp(signal_vals_pruned)]), "LPF", ),
    (np.array([iir_lowpass_interp_fill_means(signal_vals_pruned)]), "fill_means", ),
    
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)

In [ ]:
import plotly.io as pio
import plotly.graph_objects as go

def plot_signal(x, t):
    return go.Figure([go.Scatter(y=x, x=t, mode="lines+markers")
                      ]).update_layout(
        xaxis_title="Time, s",
        yaxis_title="Signal",
    )


def fft(vals, ts):
    sample_rate = ts.shape[-1]/(ts[-1] - ts[0])
    n = int(vals.shape[-1] * 13.74)
    return np.abs(np.fft.fft(vals, n=n)), np.fft.fftfreq(n)*sample_rate

def scatter_freqs(vals, ts, f_max=10):
    x, f = fft(vals, ts)
    f_idxs = np.abs(f) < f_max
    return go.Scatter(y=x[f_idxs], x=f[f_idxs], mode="lines")

def plot_freqs(vals, ts, f_max=10):
    x, f = fft(vals, ts)
    f_idxs = np.abs(f) < f_max
    return go.Figure(go.Scatter(y=x[f_idxs], x=f[f_idxs], mode="lines")).update_layout(
        xaxis_title="Frequency, Hz",
        yaxis_title="Strength",
    )



In [ ]:
def run_test(signal_vals, interval, spread, order=5):
#     signals_vals_pruned = prune_normal(signal_vals, rate, std)
    signals_vals_pruned = prune_uniform(signal_vals, interval, spread)

    interp = iir_lowpass_interp(signals_vals_pruned, order=order)
    interp2 = iir_lowpass_interp_fill_means(signals_vals_pruned, order=order)


    interp_signals = [(np.array([interp]),"LPF"), (np.array([interp]),"LPF") ]
   
    plot_freqs(signal_vals, sin_ts, 30).add_trace(
        scatter_freqs(interp,sin_ts, 30)
    ).show()
    
    return vis_signals2d(
        get_vis_df(
            sin_ts,
            np.array([signal_vals]),
            np.array([signals_vals_pruned]),
            *interp_signals,
    #         n=20000,
            freqs=[0],
    ))


In [ ]:
# run_test(signal_vals_full, 0.033, 5)
run_test(signal_vals_full, 0.1, 0.02)

In [ ]:
run_test(signal_vals_full, 0.05, 0.5)


In [ ]:
run_test(signal_vals_full, 0.27, 0)

In [ ]:
from distance_determination import estimate_dist, simulate_signals
from run_experiment import experiments
from simul.vis.signals import get_vis_df, vis_signals, vis_signals2d, get_fft_df

# import plotly.io as pio
# pio.renderers.default = "notebook_connected"
exp_start = 1000
exp_size = 5000
exp_name = "default_full"
# exp_name = "full_no_walls"
params = experiments[exp_name]
dist, signals_data_full = simulate_signals(params)
signals_data = signals_data_full[:, exp_start:exp_start+exp_size]
exp_name = "default_random"
# exp_name = "no_walls_random"
params = experiments[exp_name]
dist, signals_data_pruned_full = simulate_signals(params)
signals_data_pruned = signals_data_pruned_full[:, exp_start:exp_start+exp_size]

exp_name = "default"
# exp_name = "no_walls"
params = experiments[exp_name]
dist, signals_data_pruned_reg_full = simulate_signals(params)
signals_data_pruned_reg = signals_data_pruned_reg_full[:, exp_start:exp_start+exp_size]

In [ ]:
df = get_vis_df(
    params.tss,
    signals_data,
    signals_data_pruned,
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)
plot_freqs(signals_data_full[0,:], params.tss, 30).show()
# plot_freqs(signals_data_full[:,0], np.array(range(0, 40)), 30).show()


In [ ]:
interp = iir_lowpass_interp(signals_data_pruned[0,:], order=5)

# interp = iir_lowpass_interp_fill_means(signals_data_pruned[0,:], order=5)

interp_signals = [(np.array([interp]),"LPF")]

plot_freqs(signals_data_full[0,:], params.tss, 30).add_trace(
    scatter_freqs(interp, params.tss, 30)
).show()


vis_signals2d(
    get_vis_df(
        params.tss,
        signals_data,
        signals_data_pruned,
        *interp_signals,
#         n=20000,
        freqs=[0],
))

In [ ]:
# vals_p

interp = iir_lowpass_interp(signals_data_pruned[:,100], order=5)

# interp = iir_lowpass_interp_fill_means(signals_data_pruned[0,:], order=5)

interp_signals = [(np.array([interp]),"LPF")]

plot_freqs(signals_data_full[:,100], np.array(range(40)), 30).add_trace(
    scatter_freqs(interp, np.array(range(40)), 30)
).show()


vis_signals2d(
    get_vis_df(
        np.array(range(40)),
        np.array([signals_data[:,100]]),
        np.array([signals_data_pruned[:,100]]),
        *interp_signals,
#         n=20000,
        freqs=[0],
))